# memo

| 項目 | 英訳 | 型 |
|---|---|
| 証券コード | code | int64 |
| 企業名 | company | object |
| 会計基準 | accounting method | object |
| 連結個別 | consolidate, non-consolidate | object |
| 決算期 | settlement term, fiscal term | object |
| 決算期間 | quarter | object |
| 期首 | term beginning | datetime64[ns] |
| 期末 | term end | datetime64[ns] |
| 名寄前勘定科目 (売上高欄) | account type | object |
| 売上高 | net sales | float64 |
| 営業利益 | operating income | float64 |
| 経常利益 | ordinary income | float64 |
| 純利益 | net income | float64 |
| 一株当り純利益 | EPS, earnings per share | float64 |
| 希薄化後一株当り純利益 | diluted net income per share | float64 |
| 純資産又は株主資本 | shareholders' equity | float64 |
| 総資産 | total assets | float64 |
| 一株当り純資産 | shareholders’ equity per share | float64 |
| 営業キャッシュフロー | cash flows from operating activities | float64 |
| 投資キャッシュフロー | cash flows from investing activities | float64 |
| 財務キャッシュフロー | cash flows from financing activities | float64 |
| 情報公開日 (更新日) | publication date | datetime64[ns] |

# importなど

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import math
from dateutil.parser import parse
from sqlalchemy import create_engine
from sqlalchemy.types import Date, Integer, Float, Text

import stock

%matplotlib inline

__クラス不使用__

In [2]:
# MySQL の接続作成
db_settings = {
    "host": 'localhost',
    "database": 'StockPrice_Yahoo_1',
    "user": 'user',
    "password": 'password',
    "port":'3306'
}
engine = create_engine('mysql://{user}:{password}@{host}:{port}/{database}'.format(**db_settings))

__クラス使用__

In [120]:
sql = stock.sql() # MySQLに接続するクラスインスタンスを作成

# 結合済みファイルの読み込み

## 初回新規ファイル作成時のみの代替処理

In [56]:
kabupro_kessan = new_kessan

## MySQL からの読み込み

In [138]:
table_name = 'kabupro_kessan'

'kabupro_kessan'

__クラス不使用__

In [124]:
kabupro_kessan = pd.read_sql_table(table_name, engine, index_col=None).drop('index', axis=1)

__クラス使用__

In [132]:
kabupro_kessan = sql.read_table(table_name)

## csv からの読み込みの場合

In [ ]:
kabupro_kessan = pd.read_csv('/Users/Really/Stockyard/_csv/kabupro_kessan.csv')

In [ ]:
kabupro_kessan

csv では読み込みのたびに型を設定しなおさなければならず面倒。  
整形後に保存した csv であれば配列の型の辞書を適用可能。

In [ ]:
datatype = {'証券コード': 'int64',
                   '企業名': 'O',
                   '会計基準': 'O',
                   '連結個別': 'O',
                   '決算期': 'O',
                   '決算期間': 'O',
                   '期首': '<M8[ns]',
                   '期末': '<M8[ns]',
                   '名寄前勘定科目 (売上高欄)': 'O',
                   '売上高': 'float64',
                   '営業利益': 'float64',
                   '経常利益': 'float64',
                   '純利益': 'float64',
                   '一株当り純利益': 'float64',
                   '希薄化後一株当り純利益': 'float64',
                   '純資産又は株主資本': 'float64',
                   '総資産': 'float64',
                   '一株当り純資産': 'float64',
                   '営業キャッシュフロー': 'float64',
                   '投資キャッシュフロー': 'float64',
                   '財務キャッシュフロー': 'float64',
                   '情報公開日 (更新日)': '<M8[ns]'
                   }

In [ ]:
# 整形済みのデータが実行環境内に存在する場合
datatype = dict(new_kessan.dtypes)
datatype

In [ ]:
kabupro_kessan = kabupro_kessan.astype(datatype)

In [ ]:
kabupro_kessan.dtypes

# 新規追加ファイルの読み込み、整形、保存 

## 決算プロの決算短信 xls ファイルの読み込み

In [57]:
xls_file = '/Users/Really/Stockyard/_dl_data/20171112f.xls' # http://ke.kabupro.jp/doc/down40.htm

In [58]:
# 決算プロの決算短信xlsを読み込む
new_kessan = pd.read_excel(xls_file) 

## 内容の確認

In [59]:
new_kessan.head(20)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2018年3月期,第2四半期,2017-04-01,2017-09-30,売上高,1.204580e+11,...,1.633000e+09,155.50,144.15,2.669200e+10,1.198060e+11,NaN,NaN,NaN,NaN,2017-11-06
1,1301,極洋,日本基準,連結,2018年3月期,第1四半期,2017-04-01,2017-06-30,売上高,5.684400e+10,...,7.540000e+08,71.80,66.56,2.556000e+10,1.074220e+11,NaN,NaN,NaN,NaN,2017-08-04
2,1301,極洋,日本基準,連結,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.365610e+11,...,2.422000e+09,230.66,213.01,2.539100e+10,9.739100e+10,2378.09,6.010000e+08,-1.998000e+09,1.050000e+08,2017-05-11
3,1301,極洋,日本基準,連結,2017年3月期,第3四半期,2016-04-01,2016-12-31,売上高,1.799750e+11,...,2.449000e+09,233.26,215.13,2.577900e+10,1.171680e+11,NaN,NaN,NaN,NaN,2017-02-10
4,1301,極洋,日本基準,連結,2017年3月期,第2四半期,2016-04-01,2016-09-30,売上高,1.095700e+11,...,1.186000e+09,112.99,104.16,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-06
5,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,5.220600e+10,...,5.510000e+08,52.47,48.37,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-04
6,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,1.799000e+09,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11
7,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,1.133000e+09,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10
8,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04
9,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05


In [7]:
new_kessan.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

In [60]:
# 列名を少し変更
new_kessan.columns = [
    '証券コード',
    '企業名',
    '会計基準',
    '連結個別',
    '決算期',
    '決算期間',
    '期首',
    '期末',
    '名寄前勘定科目 (売上高欄)',
    '売上高',
    '営業利益',
    '経常利益',
    '純利益',
    '一株当り純利益',
    '希薄化後一株当り純利益',
    '純資産又は株主資本',
    '総資産',
    '一株当り純資産',
    '営業キャッシュフロー',
    '投資キャッシュフロー',
    '財務キャッシュフロー',
    '情報公開日 (更新日)'
]

In [23]:
new_kessan[['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '情報公開日 (更新日)']].head(20)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),情報公開日 (更新日)
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2015-05-08
5,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,2013-08-05
6,1301,極洋,日本基準,連結,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,2013-10-21
7,1301,極洋,日本基準,連結,2013年3月期,第3四半期,2012-04-01,2012-12-31,売上高,2014-02-05
8,1301,極洋,日本基準,連結,2013年3月期,通期,2012-04-01,2013-03-31,売上高,2014-05-09
9,1301,極洋,日本基準,連結,2014年3月期,第1四半期,2013-04-01,2013-06-30,売上高,2014-08-05


In [61]:
new_kessan.groupby('決算期').count()

,証券コード,企業名,会計基準,連結個別,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,営業利益,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
決算期,,,,,,,,,,,,,,,,,,,,,
2014年12月期,1666,1666,1666,1666,1666,1666,1666,1664,1664,1648,...,1665,1665,529,656,656,629,336,336,334,1666
2014年6月期,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
2014年9月期,6,6,6,6,6,6,6,6,6,6,...,6,6,3,5,5,4,2,2,2,6
2015年10月期,204,204,204,204,204,204,204,204,204,202,...,204,204,63,79,79,76,44,44,44,204
2015年11月期,252,252,252,252,252,252,252,252,252,248,...,251,252,44,89,89,88,48,48,48,252
2015年12月期,1830,1830,1830,1830,1830,1830,1830,1830,1830,1817,...,1830,1829,628,723,723,691,369,369,365,1830
2015年1月期,233,233,233,233,233,233,233,232,232,230,...,233,233,62,94,94,83,49,49,48,233
2015年2月期,972,972,972,972,972,972,972,971,971,962,...,972,972,264,359,359,345,194,194,194,972
2015年3月期,12169,12169,12169,12169,12169,12169,12169,12150,12150,11443,...,12166,12151,2842,4498,4498,4341,2284,2283,2278,12169


In [62]:
new_kessan.groupby('決算期間').count()

,証券コード,企業名,会計基準,連結個別,決算期,期首,期末,名寄前勘定科目 (売上高欄),売上高,営業利益,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
決算期間,,,,,,,,,,,,,,,,,,,,,
第1四半期,14128,14128,14128,14128,14128,14128,14128,14113,14113,13681,...,14125,14128,3668,4042,4042,1454,0,0,0,14128
第2四半期,14422,14422,14422,14422,14422,14422,14422,14401,14401,13450,...,14417,14417,3671,4034,4034,1428,0,0,0,14422
第3四半期,11189,11189,11189,11189,11189,11189,11189,11183,11183,10857,...,11189,11189,3198,3947,3947,1470,0,0,0,11189
通期,19230,19230,19230,19230,19230,19230,19230,19193,19193,18424,...,19227,19194,5216,17542,17542,17535,9614,9611,9577,19230


In [10]:
new_kessan[['証券コード', '企業名', '期末', '売上高', '営業利益', '経常利益', '純利益', '情報公開日 (更新日)']].head(20)

,証券コード,企業名,期末,売上高,営業利益,経常利益,純利益,情報公開日 (更新日)
0,1301,極洋,2015-03-31,2.183500e+11,2.460000e+09,2.107000e+09,2433000000,2015-05-08
1,1301,極洋,2014-12-31,1.701400e+11,2.293000e+09,1.974000e+09,2806000000,2015-02-04
2,1301,極洋,2014-09-30,1.024760e+11,7.210000e+08,6.860000e+08,2265000000,2014-11-07
3,1301,極洋,2014-06-30,4.867500e+10,3.190000e+08,3.890000e+08,223000000,2014-08-05
4,1301,極洋,2014-03-31,2.023870e+11,2.915000e+09,2.985000e+09,2968000000,2015-05-08
5,1301,極洋,2013-12-31,1.571560e+11,2.875000e+09,2.950000e+09,2724000000,2015-02-04
6,1301,極洋,2013-09-30,9.410100e+10,1.404000e+09,1.500000e+09,1880000000,2014-11-07
7,1301,極洋,2013-06-30,4.485800e+10,1.006000e+09,1.104000e+09,877000000,2014-08-05
8,1301,極洋,2013-03-31,1.780460e+11,2.324000e+09,2.262000e+09,1269000000,2014-05-09
9,1301,極洋,2012-12-31,1.377930e+11,1.824000e+09,1.960000e+09,952000000,2014-02-05


In [11]:
new_kessan[['証券コード', '企業名', '期末', '一株当り純利益', '希薄化後一株当り純利益', '純資産又は株主資本',
       '総資産', '一株当り純資産', '営業キャッシュフロー', '投資キャッシュフロー', '財務キャッシュフロー',
       '情報公開日 (更新日)']].head(20)

,証券コード,企業名,期末,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
0,1301,極洋,2015-03-31,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2015-05-08
1,1301,極洋,2014-12-31,26.72,24.63,2.360300e+10,1.068710e+11,NaN,NaN,NaN,NaN,2015-02-04
2,1301,極洋,2014-09-30,21.57,19.89,2.233800e+10,9.572100e+10,NaN,NaN,NaN,NaN,2014-11-07
3,1301,極洋,2014-06-30,2.13,1.96,1.917100e+10,9.166900e+10,NaN,NaN,NaN,NaN,2014-08-05
4,1301,極洋,2014-03-31,28.26,27.55,1.993000e+10,8.431900e+10,187.57,1.910000e+09,-1.900000e+09,-5.120000e+08,2015-05-08
5,1301,極洋,2013-12-31,25.94,25.77,NaN,NaN,NaN,NaN,NaN,NaN,2015-02-04
6,1301,極洋,2013-09-30,17.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-11-07
7,1301,極洋,2013-06-30,8.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-05
8,1301,極洋,2013-03-31,12.08,NaN,1.868300e+10,8.324500e+10,175.54,7.384000e+09,-4.690000e+08,-6.614000e+09,2014-05-09
9,1301,極洋,2012-12-31,9.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-05


In [63]:
new_kessan.dtypes

証券コード                      int64
企業名                       object
会計基準                      object
連結個別                      object
決算期                       object
決算期間                      object
期首                datetime64[ns]
期末                datetime64[ns]
名寄前勘定科目 (売上高欄)            object
売上高                      float64
営業利益                     float64
経常利益                     float64
純利益                      float64
一株当り純利益                  float64
希薄化後一株当り純利益              float64
純資産又は株主資本                float64
総資産                      float64
一株当り純資産                  float64
営業キャッシュフロー               float64
投資キャッシュフロー               float64
財務キャッシュフロー               float64
情報公開日 (更新日)       datetime64[ns]
dtype: object

In [ ]:
# 2015年のファイルでは int
new_kessan['純利益'].max()

In [ ]:
# 2017年のファイルでは float
kabupro_kessan_new['純利益'].max()

In [ ]:
# 2015年のファイルでは文字列が混入しているため object
new_kessan['希薄化後一株当り純利益'].max()

In [ ]:
new_kessan['希薄化後一株当り純利益'].min()

In [ ]:
new_kessan['期末'].min()

In [ ]:
new_kessan['情報公開日 (更新日)'].min()

In [64]:
new_kessan.duplicated().any()

False

In [65]:
new_kessan.isnull().any()

証券コード             False
企業名               False
会計基準              False
連結個別              False
決算期               False
決算期間              False
期首                False
期末                False
名寄前勘定科目 (売上高欄)     True
売上高                True
営業利益               True
経常利益               True
純利益                True
一株当り純利益            True
希薄化後一株当り純利益        True
純資産又は株主資本          True
総資産                True
一株当り純資産            True
営業キャッシュフロー         True
投資キャッシュフロー         True
財務キャッシュフロー         True
情報公開日 (更新日)       False
dtype: bool

## 整形

In [17]:
# 2015年のファイルでは int 型だったので float 型に
new_kessan['純利益'] = new_kessan['純利益'].astype(float)

In [20]:
# 2015年のファイルでは index 17979 に '―' が入っていたために列が object 型になってしまっている
new_kessan[new_kessan['希薄化後一株当り純利益'].apply(lambda x: type(x) is str)]

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)


In [19]:
# '―'  を NaN に置換
new_kessan.loc[new_kessan['希薄化後一株当り純利益'].apply(lambda x: type(x) is str), '希薄化後一株当り純利益'] = np.nan

In [ ]:
new_kessan['希薄化後一株当り純利益'][17979]

In [ ]:
new_kessan['希薄化後一株当り純利益'].max()

In [21]:
# float 型に変換
new_kessan['希薄化後一株当り純利益'] = new_kessan['希薄化後一株当り純利益'].astype(float)

In [66]:
# ソート
new_kessan = new_kessan.sort_values(['証券コード', '連結個別', '期末', '情報公開日 (更新日)']).reset_index(drop=True)

## csv に保存

In [111]:
# 保存
new_kessan.to_csv('/Users/Really/Stockyard/_csv/kabupro_kessan_20171112.csv')

In [117]:
# 保存したファイルの確認
csv_kessan = pd.read_csv('/Users/Really/Stockyard/_csv/kabupro_kessan_20171112.csv', index_col=0)
csv_kessan.head(20)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
0,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
1,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
2,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
3,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
4,1301,極洋,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,4.867500e+10,...,2.230000e+08,2.13,1.96,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-03
5,1301,極洋,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,1.024760e+11,...,2.265000e+09,21.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
6,1301,極洋,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,1.701400e+11,...,2.806000e+09,26.72,24.63,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05
7,1301,極洋,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.183500e+11,...,2.433000e+09,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2016-05-09
8,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05
9,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04


## MySQL に保存

In [68]:
table_name = 'kabupro_kessan_20171112'

このケースではデータ型の定義は必要ないのかも  
定義しなくても意図通りの型で書き込めた

In [69]:
# データ型を定義しなくても意図通りの型で書き込めた
new_kessan.to_sql(table_name, engine, if_exists='replace')

In [ ]:
# このケースではデータ型の定義は必要ないのかも
# sqlalchemy.typesで定義されたデータ型を辞書形式で設定して保存
data_type = {
    '`証券コード`': Integer(),
    '`企業名`': Text(),
    '`会計基準`': Text(),
    '`連結個別`': Text(),
    '`決算期`': Text(),
    '`決算期間`': Text(),
    '`期首`': Date(),
    '`期末`': Date(),
    '`名寄前勘定科目 (売上高欄)`': Text(),
    '`売上高`': Float(),
    '`営業利益`': Float(),
    '`経常利益`': Float(),
    '`純利益`': Float(),
    '`一株当り純利益`': Float(),
    '`希薄化後一株当り純利益`': Float(),
    '`純資産又は株主資本`': Float(),
    '`総資産`': Float(),
    '`一株当り純資産`': Float(),
    '`営業キャッシュフロー`': Float(),
    '`投資キャッシュフロー`': Float(),
    '`財務キャッシュフロー`': Float(),
    '`情報公開日 (更新日)`': Date()
}

kabupro_kessan.to_sql(table_name, engine, if_exists='replace', dtype=data_type)

__保存したデータの確認__

In [70]:
sql_kessan = pd.read_sql_table(table_name, engine, index_col=None).drop('index', axis=1)

In [73]:
sql_kessan.head(20)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
0,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
1,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
2,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
3,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
4,1301,極洋,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,4.867500e+10,...,2.230000e+08,2.13,1.96,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-03
5,1301,極洋,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,1.024760e+11,...,2.265000e+09,21.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
6,1301,極洋,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,1.701400e+11,...,2.806000e+09,26.72,24.63,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05
7,1301,極洋,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.183500e+11,...,2.433000e+09,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2016-05-09
8,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05
9,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04


In [71]:
sql_kessan.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

In [72]:
sql_kessan.dtypes

証券コード                      int64
企業名                       object
会計基準                      object
連結個別                      object
決算期                       object
決算期間                      object
期首                datetime64[ns]
期末                datetime64[ns]
名寄前勘定科目 (売上高欄)            object
売上高                      float64
営業利益                     float64
経常利益                     float64
純利益                      float64
一株当り純利益                  float64
希薄化後一株当り純利益              float64
純資産又は株主資本                float64
総資産                      float64
一株当り純資産                  float64
営業キャッシュフロー               float64
投資キャッシュフロー               float64
財務キャッシュフロー               float64
情報公開日 (更新日)       datetime64[ns]
dtype: object

In [74]:
# nan 同士の比較は False になってしまうのでゼロで埋めて比較
(sql_kessan.fillna(0) == new_kessan.fillna(0)).any()

証券コード             True
企業名               True
会計基準              True
連結個別              True
決算期               True
決算期間              True
期首                True
期末                True
名寄前勘定科目 (売上高欄)    True
売上高               True
営業利益              True
経常利益              True
純利益               True
一株当り純利益           True
希薄化後一株当り純利益       True
純資産又は株主資本         True
総資産               True
一株当り純資産           True
営業キャッシュフロー        True
投資キャッシュフロー        True
財務キャッシュフロー        True
情報公開日 (更新日)       True
dtype: bool

# 新旧ファイルの結合

In [75]:
kabupro_kessan = kabupro_kessan.append(new_kessan)

In [105]:
kabupro_kessan.tail(20)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
110390,9997,ベルーナ,日本基準,連結,2014年3月期,第3四半期,2013-04-01,2013-12-31,売上高,9.601400e+10,...,5.424000e+09,55.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-30
110391,9997,ベルーナ,日本基準,連結,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.254120e+11,...,7.013000e+09,72.12,NaN,7.348000e+10,1.306480e+11,755.67,5.766000e+09,-1.539700e+10,1.072100e+10,2015-05-15
110392,9997,ベルーナ,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,3.306800e+10,...,8.740000e+08,8.99,NaN,7.383800e+10,1.342050e+11,NaN,NaN,NaN,NaN,2014-07-31
110393,9997,ベルーナ,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,3.306800e+10,...,8.740000e+08,8.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-31
110394,9997,ベルーナ,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,5.631400e+10,...,2.051000e+09,21.09,NaN,7.515800e+10,1.360600e+11,NaN,NaN,NaN,NaN,2014-10-31
110395,9997,ベルーナ,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,5.631400e+10,...,2.051000e+09,21.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-30
110396,9997,ベルーナ,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,9.268800e+10,...,5.397000e+09,55.51,NaN,7.814300e+10,1.551150e+11,NaN,NaN,NaN,NaN,2015-01-30
110397,9997,ベルーナ,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,9.268800e+10,...,5.397000e+09,55.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-29
110398,9997,ベルーナ,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,1.206890e+11,...,6.394000e+09,65.77,NaN,7.951000e+10,1.522240e+11,814.97,9.089000e+09,-2.019100e+10,1.257100e+10,2015-05-15
110399,9997,ベルーナ,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,1.206890e+11,...,6.394000e+09,65.77,NaN,7.951000e+10,1.522240e+11,814.97,9.089000e+09,-2.019100e+10,1.257100e+10,2016-05-11


In [79]:
kabupro_kessan.dtypes

証券コード                      int64
企業名                       object
会計基準                      object
連結個別                      object
決算期                       object
決算期間                      object
期首                datetime64[ns]
期末                datetime64[ns]
名寄前勘定科目 (売上高欄)            object
売上高                      float64
営業利益                     float64
経常利益                     float64
純利益                      float64
一株当り純利益                  float64
希薄化後一株当り純利益              float64
純資産又は株主資本                float64
総資産                      float64
一株当り純資産                  float64
営業キャッシュフロー               float64
投資キャッシュフロー               float64
財務キャッシュフロー               float64
情報公開日 (更新日)       datetime64[ns]
dtype: object

In [77]:
# ソート
kabupro_kessan = kabupro_kessan.sort_values(['証券コード', '連結個別', '期末', '情報公開日 (更新日)']).reset_index(drop=True)

## 重複の確認、削除

In [106]:
kabupro_kessan.duplicated().any()

False

In [107]:
# keep=Falseで重複行のいずれかではなくすべてを表示
kabupro_kessan[kabupro_kessan.duplicated(keep=False)]

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)


In [82]:
kabupro_kessan = kabupro_kessan.drop_duplicates()

In [108]:
# インデックスの振り直し
kabupro_kessan = kabupro_kessan.reset_index(drop=True)

## 内容の確認

In [84]:
kabupro_kessan.isnull().any()

証券コード             False
企業名               False
会計基準              False
連結個別              False
決算期               False
決算期間              False
期首                False
期末                False
名寄前勘定科目 (売上高欄)     True
売上高                True
営業利益               True
経常利益               True
純利益                True
一株当り純利益            True
希薄化後一株当り純利益        True
純資産又は株主資本          True
総資産                True
一株当り純資産            True
営業キャッシュフロー         True
投資キャッシュフロー         True
財務キャッシュフロー         True
情報公開日 (更新日)       False
dtype: bool

In [85]:
kabupro_kessan['期末'].min()

Timestamp('2012-02-20 00:00:00')

In [86]:
kabupro_kessan['情報公開日 (更新日)'].min()

Timestamp('2012-04-26 00:00:00')

In [87]:
kabupro_kessan['期末'].max()

Timestamp('2017-09-30 00:00:00')

In [88]:
kabupro_kessan['情報公開日 (更新日)'].max()

Timestamp('2017-11-10 00:00:00')

In [ ]:
kabupro_kessan.describe()

## csv に保存

In [118]:
# 保存
kabupro_kessan.to_csv('/Users/Really/Stockyard/_csv/kabupro_kessan.csv')

In [119]:
# 保存したファイルの確認
csv_kessan = pd.read_csv('/Users/Really/Stockyard/_csv/kabupro_kessan.csv', index_col=0)
csv_kessan.head(20)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,7.654600e+10,...,7.900000e+07,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.606100e+11,...,8.060000e+08,7.68,NaN,1.481800e+10,7.049200e+10,141.08,NaN,NaN,NaN,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,8.816100e+10,...,1.834000e+09,17.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.920260e+11,...,1.806000e+09,17.20,16.77,1.616400e+10,7.038100e+10,153.90,NaN,NaN,NaN,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2015-05-08
6,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
7,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
8,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
9,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,4.319100e+10,...,3.350000e+08,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-05


## MySQL に保存

In [91]:
table_name = 'kabupro_kessan'

このケースではデータ型の定義は必要ないのかも  
定義しなくても意図通りの型で書き込めた

__クラス不使用__

In [92]:
# データ型を定義しなくても意図通りの型で書き込めた
kabupro_kessan.to_sql(table_name, engine, if_exists='replace')

In [ ]:
# このケースではデータ型の定義は必要ないのかも
# sqlalchemy.typesで定義されたデータ型を辞書形式で設定して保存
data_type = {
    '`証券コード`': Integer(),
    '`企業名`': Text(),
    '`会計基準`': Text(),
    '`連結個別`': Text(),
    '`決算期`': Text(),
    '`決算期間`': Text(),
    '`期首`': Date(),
    '`期末`': Date(),
    '`名寄前勘定科目 (売上高欄)`': Text(),
    '`売上高`': Float(),
    '`営業利益`': Float(),
    '`経常利益`': Float(),
    '`純利益`': Float(),
    '`一株当り純利益`': Float(),
    '`希薄化後一株当り純利益`': Float(),
    '`純資産又は株主資本`': Float(),
    '`総資産`': Float(),
    '`一株当り純資産`': Float(),
    '`営業キャッシュフロー`': Float(),
    '`投資キャッシュフロー`': Float(),
    '`財務キャッシュフロー`': Float(),
    '`情報公開日 (更新日)`': Date()
}

kabupro_kessan.to_sql(table_name, engine, if_exists='replace', dtype=data_type)

__クラス使用__

In [123]:
sql.write_table(table_name, kabupro_kessan)

__保存したデータの確認__

__クラス不使用__

In [124]:
sql_kessan = pd.read_sql_table(table_name, engine, index_col=None).drop('index', axis=1)

__クラス使用__

In [132]:
sql_kessan = sql.read_table(table_name)

In [128]:
sql_kessan.head(20)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,7.654600e+10,...,7.900000e+07,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.606100e+11,...,8.060000e+08,7.68,NaN,1.481800e+10,7.049200e+10,141.08,NaN,NaN,NaN,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,8.816100e+10,...,1.834000e+09,17.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.920260e+11,...,1.806000e+09,17.20,16.77,1.616400e+10,7.038100e+10,153.90,NaN,NaN,NaN,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2015-05-08
6,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
7,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
8,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
9,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,4.319100e+10,...,3.350000e+08,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-05


In [129]:
sql_kessan.tail(20)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
109506,9997,ベルーナ,日本基準,連結,2014年3月期,第3四半期,2013-04-01,2013-12-31,売上高,9.601400e+10,...,5.424000e+09,55.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-30
109507,9997,ベルーナ,日本基準,連結,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.254120e+11,...,7.013000e+09,72.12,NaN,7.348000e+10,1.306480e+11,755.67,5.766000e+09,-1.539700e+10,1.072100e+10,2015-05-15
109508,9997,ベルーナ,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,3.306800e+10,...,8.740000e+08,8.99,NaN,7.383800e+10,1.342050e+11,NaN,NaN,NaN,NaN,2014-07-31
109509,9997,ベルーナ,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,3.306800e+10,...,8.740000e+08,8.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-31
109510,9997,ベルーナ,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,5.631400e+10,...,2.051000e+09,21.09,NaN,7.515800e+10,1.360600e+11,NaN,NaN,NaN,NaN,2014-10-31
109511,9997,ベルーナ,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,5.631400e+10,...,2.051000e+09,21.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-30
109512,9997,ベルーナ,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,9.268800e+10,...,5.397000e+09,55.51,NaN,7.814300e+10,1.551150e+11,NaN,NaN,NaN,NaN,2015-01-30
109513,9997,ベルーナ,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,9.268800e+10,...,5.397000e+09,55.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-29
109514,9997,ベルーナ,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,1.206890e+11,...,6.394000e+09,65.77,NaN,7.951000e+10,1.522240e+11,814.97,9.089000e+09,-2.019100e+10,1.257100e+10,2015-05-15
109515,9997,ベルーナ,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,1.206890e+11,...,6.394000e+09,65.77,NaN,7.951000e+10,1.522240e+11,814.97,9.089000e+09,-2.019100e+10,1.257100e+10,2016-05-11


In [100]:
sql_kessan.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

In [95]:
sql_kessan.dtypes

証券コード                      int64
企業名                       object
会計基準                      object
連結個別                      object
決算期                       object
決算期間                      object
期首                datetime64[ns]
期末                datetime64[ns]
名寄前勘定科目 (売上高欄)            object
売上高                      float64
営業利益                     float64
経常利益                     float64
純利益                      float64
一株当り純利益                  float64
希薄化後一株当り純利益              float64
純資産又は株主資本                float64
総資産                      float64
一株当り純資産                  float64
営業キャッシュフロー               float64
投資キャッシュフロー               float64
財務キャッシュフロー               float64
情報公開日 (更新日)       datetime64[ns]
dtype: object

In [102]:
kabupro_kessan.dtypes

証券コード                      int64
企業名                       object
会計基準                      object
連結個別                      object
決算期                       object
決算期間                      object
期首                datetime64[ns]
期末                datetime64[ns]
名寄前勘定科目 (売上高欄)            object
売上高                      float64
営業利益                     float64
経常利益                     float64
純利益                      float64
一株当り純利益                  float64
希薄化後一株当り純利益              float64
純資産又は株主資本                float64
総資産                      float64
一株当り純資産                  float64
営業キャッシュフロー               float64
投資キャッシュフロー               float64
財務キャッシュフロー               float64
情報公開日 (更新日)       datetime64[ns]
dtype: object

In [133]:
# nan 同士の比較は False になってしまうのでゼロで埋めて比較
(sql_kessan.fillna(0) == kabupro_kessan.fillna(0)).any()

証券コード             True
企業名               True
会計基準              True
連結個別              True
決算期               True
決算期間              True
期首                True
期末                True
名寄前勘定科目 (売上高欄)    True
売上高               True
営業利益              True
経常利益              True
純利益               True
一株当り純利益           True
希薄化後一株当り純利益       True
純資産又は株主資本         True
総資産               True
一株当り純資産           True
営業キャッシュフロー        True
投資キャッシュフロー        True
財務キャッシュフロー        True
情報公開日 (更新日)       True
dtype: bool

# 別アプリで csv に変換済みのファイルを読み込む場合

In [ ]:
# 決算プロの決算短信csv(エクセルファイルより書き出し)を読み込む http://ke.kabupro.jp/doc/down40.htm
csv_version = pd.read_csv('/Users/Really/Stockyard/_dl_data/kabupro_20171112f.csv') 

## 内容の確認

In [ ]:
csv_version

csv からの読み込みではほとんどの列の型が object になる (読み込み時にオプションで指定することも可能らしいが、列が多いと面倒)

In [ ]:
csv_version.dtypes

In [ ]:
csv_version.duplicated().any()

In [ ]:
csv_version.isnull().any()

In [ ]:
csv_version['期末'].min()

In [ ]:
csv_version['情報公開日 (更新日)'].min()

In [ ]:
csv_version[csv_version['連結個別'] == '個別']

In [ ]:
csv_version[csv_version['連結個別'] == '連結']

In [ ]:
csv_version.describe()

## 整形

In [ ]:
# 日付のパース、datetime.dateへの型変換
csv_version['期首'] = csv_version['期首'].apply(lambda x: parse(x).date())
csv_version['期末'] = csv_version['期末'].apply(lambda x: parse(x).date())
csv_version['情報公開日 (更新日)'] = csv_version['情報公開日 (更新日)'].apply(lambda x: parse(x).date())

In [ ]:
# 同じく日付のパース、datetime.dateへの型変換をまとめて。1列ずつやるより少し遅い？ 23s -> 30s
csv_version[['期首', '期末', '情報公開日 (更新日)']] = csv_version[['期首', '期末', '情報公開日 (更新日)']].applymap(lambda x: parse(x).date())

In [ ]:
# pandasのTimestampへの型変換
# まとめてできないの？
# csv_version[['期首', '期末', '情報公開日 (更新日)']] = pd.to_datetime(csv_version[['期首', '期末', '情報公開日 (更新日)']], format='%Y-%m-%d')
csv_version['期首'] = pd.to_datetime(csv_version['期首'], format='%Y-%m-%d')
csv_version['期末'] = pd.to_datetime(csv_version['期末'], format='%Y-%m-%d')
csv_version['情報公開日 (更新日)'] = pd.to_datetime(csv_version['情報公開日 (更新日)'], format='%Y-%m-%d')

In [ ]:
# 数値に変換する列のリスト作成
num_list = [
    '売上高', 
    '営業利益', 
    '経常利益', 
    '純利益', 
    '一株当り純利益',
    '希薄化後一株当り純利益', 
    '純資産又は株主資本', 
    '総資産', 
    '一株当り純資産', 
    '営業キャッシュフロー',
    '投資キャッシュフロー', 
    '財務キャッシュフロー'
]

In [ ]:
# 数値に変換する項目の「,」を削除、float型に変換
csv_version[num_list] = csv_version[num_list].apply(lambda x: x.str.replace(',','')).astype(float)

In [ ]:
# 冗長なやり方

# 2つのリストから1つの辞書を作成する方法
# num_dict = dict(zip(num_list, [str for i in range(len(num_list))]))

csv_version = csv_version.astype(dict(zip(num_list, [str for i in range(len(num_list))])))
csv_version[num_list] = csv_version[num_list].applymap(lambda x: x.replace(',', ''))
csv_version = csv_version.astype(dict(zip(num_list, [float for i in range(len(num_list))])))

In [ ]:
# さらに冗長なやり方

csv_version = csv_version.astype({
    '売上高': str, 
    '営業利益': str, 
    '経常利益': str, 
    '純利益': str, 
    '一株当り純利益': str,
    '希薄化後一株当り純利益': str, 
    '純資産又は株主資本': str, 
    '総資産': str, 
    '一株当り純資産': str, 
    '営業キャッシュフロー': str,
    '投資キャッシュフロー': str, 
    '財務キャッシュフロー': str})

csv_version[[
    '売上高', 
    '営業利益', 
    '経常利益', 
    '純利益', 
    '一株当り純利益',
    '希薄化後一株当り純利益', 
    '純資産又は株主資本', 
    '総資産', 
    '一株当り純資産', 
    '営業キャッシュフロー',
    '投資キャッシュフロー', 
    '財務キャッシュフロー'
    ]] = csv_version[[
        '売上高', 
        '営業利益', 
        '経常利益', 
        '純利益', 
        '一株当り純利益',
        '希薄化後一株当り純利益', 
        '純資産又は株主資本', 
        '総資産', 
        '一株当り純資産', 
        '営業キャッシュフロー',
        '投資キャッシュフロー', 
        '財務キャッシュフロー'
        ]].applymap(lambda x: x.replace(',', ''))

csv_version = csv_version.astype({
    '売上高': float, 
    '営業利益': float, 
    '経常利益': float, 
    '純利益': float, 
    '一株当り純利益': float,
    '希薄化後一株当り純利益': float, 
    '純資産又は株主資本': float, 
    '総資産': float, 
    '一株当り純資産': float, 
    '営業キャッシュフロー': float,
       '投資キャッシュフロー': float, 
    '財務キャッシュフロー': float})

In [ ]:
# ソート
csv_version = csv_version.sort_values(['証券コード', '連結個別', '期末', '情報公開日 (更新日)']).reset_index(drop=True)

# 四半期業績の差分を作ってみる

__もうちょっと上手いやり方ありそう__

In [ ]:
code = 7203

In [ ]:
kabupro.columns

In [ ]:
kabupro.ix[(kabupro['証券コード'] == code) & (kabupro['会計基準'] == '米国基準'), 
           ['決算期', '期末', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益', '情報公開日 (更新日)']].tail(10)

In [ ]:
diff_test = kabupro.ix[(kabupro['証券コード'] == code) & (kabupro['会計基準'] == '米国基準'), 
           ['決算期', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益']].tail(10)

In [ ]:
diff_test

In [ ]:
diff_test[['売上高差分', '営業利益差分', '経常利益差分', '純利益差分', '一株当り純利益差分']] = diff_test[['売上高', '営業利益', '経常利益', '純利益', '一株当り純利益']]

In [ ]:
# 全銘柄一括でやるなら、上の行とコードが一致するかの判定を追加
for count in range(70501, 70501 + len(diff_test) - 1):
    if diff_test.loc[count, '決算期'] == diff_test.loc[count - 1, '決算期']:
        diff_test.loc[count, '売上高差分'] = diff_test.loc[count, '売上高'] - diff_test.loc[count - 1, '売上高']
        diff_test.loc[count, '営業利益差分'] = diff_test.loc[count, '営業利益'] - diff_test.loc[count - 1, '営業利益']
        diff_test.loc[count, '経常利益差分'] = diff_test.loc[count, '経常利益'] - diff_test.loc[count - 1, '経常利益']
        diff_test.loc[count, '純利益差分'] = diff_test.loc[count, '純利益'] - diff_test.loc[count - 1, '純利益']
        diff_test.loc[count, '一株当り純利益差分'] = diff_test.loc[count, '一株当り純利益'] - diff_test.loc[count - 1, '一株当り純利益']

In [ ]:
diff_test[['決算期', '売上高差分', '営業利益差分', '経常利益差分', '純利益差分', '一株当り純利益差分']]
# 一株当り純利益差分が株探の１株益と揃わない